In [104]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
import numpy as np
import os,glob,cv2
from os.path import dirname
import sys,argparse
from sklearn.utils import shuffle

In [71]:
image_size=128
num_channels=3
label =[]

In [154]:
files = []
images = []
labels = []
testdata_path="./data/potholes/Test-Data/"
for (dirpath, dirnames, filenames) in os.walk(testdata_path):
    print(dirnames)
    for file in filenames:
        imagefile = os.path.join(dirpath, file)
        if (os.path.isdir(imagefile)):
            continue
        image = cv2.imread(imagefile)
        image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
        image = image.astype(np.float32)
        image = np.multiply(image, 1.0 / 255.0)
        images.append(image)
        folder = os.path.basename(dirpath)
        if (folder == 'Negative'):
            label = [1,0]
        if (folder == 'Positive'):
            label =[0,1]
        labels.append(np.asarray(label))
images = np.array(images)
labels = np.array(labels)
shuffle(images, labels)


['Positive', 'Negative']
[]
[]


[array([[[[0.27450982, 0.5137255 , 0.5294118 ],
          [0.34901962, 0.5764706 , 0.5803922 ],
          [0.49803925, 0.49803925, 0.48627454],
          ...,
          [0.854902  , 0.63529414, 0.41960788],
          [0.8431373 , 0.62352943, 0.40784317],
          [0.8470589 , 0.627451  , 0.41176474]],
 
         [[0.56078434, 0.6901961 , 0.6901961 ],
          [0.3529412 , 0.58431375, 0.57254905],
          [0.20000002, 0.37254903, 0.36862746],
          ...,
          [0.86666673, 0.6509804 , 0.427451  ],
          [0.86666673, 0.64705884, 0.43529415],
          [0.85098046, 0.63529414, 0.41176474]],
 
         [[0.32156864, 0.6156863 , 0.6392157 ],
          [0.19215688, 0.4039216 , 0.4039216 ],
          [0.1764706 , 0.3803922 , 0.3803922 ],
          ...,
          [0.86666673, 0.64705884, 0.4156863 ],
          [0.854902  , 0.6392157 , 0.41176474],
          [0.85098046, 0.63529414, 0.41176474]],
 
         ...,
 
         [[0.34117648, 0.34509805, 0.3254902 ],
          [0.27843

In [163]:
print(labels)

[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


In [155]:
x_batch = images.reshape(len(labels), image_size,image_size,num_channels)
x_labels = labels.reshape(len(labels), 2)

In [156]:
## Let us restore the saved model 
sess = tf.Session()
# Step-1: Recreate the network graph. At this step only graph is created.
saver = tf.train.import_meta_graph('./SavedModel/multiclass_potholes_model.meta')
# Step-2: Now let's load the weights saved using the restore method.
saver.restore(sess, tf.train.latest_checkpoint('./SavedModel/'))

INFO:tensorflow:Restoring parameters from ./SavedModel/multiclass_potholes_model


In [157]:
# Accessing the default graph which we have restored
graph = tf.get_default_graph()

# Now, let's get hold of the op that we can be processed to get the output.
# In the original network y_pred is the tensor that is the prediction of the network
y_pred = graph.get_tensor_by_name("y_pred:0")

## Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
y_test_images = np.zeros((1, 2)) 

In [158]:
### Creating the feed_dict that is required to be fed to calculate y_pred 
feed_dict_testing = {x: x_batch, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
# result is of this format [probabiliy_of_rose probability_of_sunflower]
print(result)

[[0.03428876 0.96571124]
 [0.02425201 0.975748  ]
 [0.02989602 0.9701039 ]
 [0.01194796 0.988052  ]
 [0.01387409 0.9861259 ]
 [0.03844582 0.96155417]
 [0.031732   0.968268  ]
 [0.03446278 0.96553725]
 [0.02802388 0.97197616]
 [0.0472468  0.9527532 ]
 [0.03486803 0.965132  ]
 [0.02175088 0.9782491 ]
 [0.04464867 0.95535135]
 [0.05045214 0.9495479 ]
 [0.4666878  0.53331226]
 [0.01629788 0.98370206]
 [0.02456634 0.9754336 ]
 [0.03596954 0.9640304 ]
 [0.02121935 0.9787806 ]
 [0.02747411 0.97252595]
 [0.05858733 0.9414126 ]
 [0.02068738 0.97931266]
 [0.01923133 0.98076874]
 [0.03301591 0.96698403]
 [0.10763646 0.89236355]
 [0.02100084 0.97899914]
 [0.03238932 0.9676107 ]
 [0.01874715 0.98125285]
 [0.02397296 0.9760271 ]
 [0.02162216 0.9783779 ]
 [0.02068068 0.9793193 ]
 [0.02025138 0.97974867]
 [0.03259674 0.96740323]
 [0.03990124 0.9600988 ]
 [0.4288847  0.57111526]
 [0.5390015  0.46099845]
 [0.78210807 0.21789193]
 [0.8050268  0.19497319]
 [0.9046658  0.09533421]
 [0.7153577  0.2846423 ]


In [168]:
auc, update_op = tf.metrics.auc(
    labels,
    predictions
)
print(auc)

Tensor("auc_13/value:0", shape=(), dtype=float32)


In [169]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("tf auc: {}".format(sess.run([auc, update_op])))

tf auc: [0.0, 0.011966708]


In [172]:
auc = tf.contrib.metrics.streaming_auc(result, labels)

sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(tf.initialize_local_variables()) # try commenting this line and you'll get the error
train_auc = sess.run(auc)

print(train_auc)

(0.037634417, 0.012082327)


In [174]:
n_items = labels.size
accuracy = (labels ==  predictions).sum() / n_items
print("Accuracy :", accuracy)

AttributeError: 'bool' object has no attribute 'sum'

In [177]:
auc, update_op = tf.metrics.auc(
    labels,
    predictions,
    weights=None,
    num_thresholds=10,
    metrics_collections=None,
    updates_collections=None,
    curve='ROC',
    name=None,
    summation_method='trapezoidal'
)

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("tf auc: {}".format(sess.run([auc, update_op])))

tf auc: [0.0, 0.009480875]
